In [1]:
import numpy as np
from fractions import Fraction

def solution(m):
    # General strategy:
    # For each terminal state, make a system of equations using the recurrence relation for each state
    
    # P(s0->st) = P(s0->s0)*P(s0->st) + P(s0->s1)*P(s1->st) + ... + P(s0->sn)*P(sn->st)
    # P(s1->st) = P(s1->s0)*P(s0->st) + P(s1->s1)*P(s1->st) + ... + P(s1->sn)*P(sn->st)
    # ...
    # P(sn->st) = P(sn->s0)*P(s0->st) + ... + P(sn->sn)*P(sn->st)
    
    m_np = np.array(m)
    
    # Get the terminal states: P(si->si) = 1 <--> all of weight into diagonals
    terminal_states = []
    for i in range(m_np.shape[0]):
        if m_np[i, i].astype(int) == m_np[i].sum().astype(int):
            terminal_states.append(i)
            
    probs = np.array([])
    if 0 in terminal_states:
        for t in terminal_states:
            if t == 0:
                probs = np.append(probs, 1)
            else:
                probs = np.append(probs, 0)
        probs = np.append(probs, 1)
        return list(probs.astype(int))
    
    for t in terminal_states:
        if t == 0:
            continue
        m_np[t, :] = np.zeros( (1, m_np.shape[0]) ).astype(int)
    
    for t in terminal_states:
        rref_matrix = rref(create_matrix(m_np, t)) # first line of the rref matrix tells you P(s0->s_target)
        probs = np.append(probs, rref_matrix[0][-1])
        # print(np.array(rref_matrix).astype(float))
    
    # Convert our probabilities into the right format
    if len(probs) >= 2:
        denom_lcm = int(max([lcm(probs[i].denominator, probs[i+1].denominator) for i in range(len(probs) - 1)]))
    else:
        denom_lcm = int(probs[0])
    ans = []
    ans = [int(denom_lcm / f.denominator * f.numerator) for f in probs]
    ans.append(denom_lcm)
    
    return ans

def rref(B):
    # https://gist.github.com/sgsfak/77a1c08ac8a9b0af77393b24e44c9547
    A = B.copy()
    rows, cols = A.shape
    r = 0
    pivots_pos = []
    row_exchanges = np.arange(rows)
    for c in range(cols):
        
        # Find the pivot row:
        pivot = np.argmax (np.abs (A[r:rows, c])) + r
        m = np.abs(A[pivot, c])
        if m <= 1e-8:
            # Skip column c, making sure the approximately zero terms are actually zero.
            A[r:rows, c] = np.zeros(rows - r)
        else:
            # Keep track of bound variables
            pivots_pos.append((r, c))
            if pivot != r:
                A[[pivot, r], c:cols] = A[[r, pivot], c:cols]
                row_exchanges[[pivot, r]] = row_exchanges[[r, pivot]]
                
            # Normalize pivot row
            A[r, c:cols] = A[r, c:cols] / A[r, c]
            
            # Eliminate the current column
            v = A[r, c:cols]
            # Above (below row r):
            if r > 0:
                ridx_above = np.arange(r)
                A[ridx_above, c:cols] = A[ridx_above, c:cols] - np.outer(v, A[ridx_above, c]).T
                
            # Below (after row r):
            if r < rows - 1:
                ridx_below = np.arange(r + 1, rows)
                A[ridx_below, c:cols] = A[ridx_below, c:cols] - np.outer(v, A[ridx_below, c]).T
                
            r += 1
            
        # Check if done
        if r == rows:
            break;
        
    return A

def gcd(x, y):
   
    while(y):
        x, y = y, x % y
    return abs(x)

def lcm(x, y):
    return x * y / gcd(x, y)

def normalize(array):
    # Also functions as a Fraction converter
    s = sum(array)
    if s <= 0:
        s = 1
    return np.array(list(map(lambda x: Fraction(x, s), array)))

def create_matrix(experiment, target_state):
    # Get the system of equations into a form of: Ax = b
    experiment = np.array(experiment)
    state_count = experiment.shape[0]

    # A
    isolated_vars = np.apply_along_axis(normalize, 1, experiment) - np.eye(state_count).astype(int)

    # b
    b = np.zeros((state_count, 1)).astype(int)
    b[target_state] = -1
    b = np.apply_along_axis(normalize, 1, b)

    augmented_matrix = np.hstack((isolated_vars, b))
    # augmented_matrix = np.vstack((augmented_matrix, ones))

    return augmented_matrix

In [2]:
# test 1
m = [
    [0, 1, 0, 0, 0, 1],
    [4, 0, 0, 3, 2, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0]
]
solution(m)

[0, 3, 2, 9, 14]

In [3]:
# test 2
m = [
[1, 1, 1, 1, 1,],
[0, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,],
[0, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,]
]
solution(m)

[1, 1, 2]

In [4]:
# test 3
m = [
[1, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,],
[1, 1, 1, 1, 1,],
[0, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,]
]
solution(m)

[1, 0, 1]

In [5]:
# test 4
# terminal states:
# s0, s3
m = [
[100, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,],
[1, 1, 1, 1, 1,],
[0, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,]
]
solution(m)

[1, 0, 1]

In [6]:
# test 5
m = [
[1, 1, 0, 0, 0,], 
[0, 1, 0, 0, 0,],
[1, 1, 1, 1, 1,],
[0, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,]
]
solution(m)
# should be impossible to get to state 3 from state 0, so rref has no solutions for it
# regardless, it is a terminal state

[1, 0, 1]

In [7]:
# test 6
# s1, s3
# [1, 1, 2] by inspection
# state 1 effectively brings you to state 2. either loop back to a "state 2 - like" scenario or hop onto either
m = [
[1, 0, 1, 0, 0,], 
[0, 1, 0, 0, 0,],
[1, 1, 1, 1, 1,],
[0, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,]
]
solution(m)

[1, 1, 2]

In [8]:
# test 7
# [0, 1, 0 1]
# s1, s2, s3 are the terminal states
# s1 and s3 impossiblle to get to from s0
m = [
[1, 0, 1, 0, 0,], 
[0, 0, 0, 0, 0,],
[0, 0, 1, 0, 0,],
[0, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,]
]
solution(m)

[0, 1, 0, 1]

In [9]:
# test 8
# terminal states:
# s1, s2, s3
# [0, 0, 1, 1]
m = [
[1, 0, 0, 1, 0,], 
[0, 0, 0, 0, 0,],
[0, 0, 1, 0, 0,],
[0, 0, 0, 0, 0,],
[1, 1, 1, 1, 1,]
]
solution(m)

[0, 0, 1, 1]

In [10]:
# test 9
m = [
[0]
]
solution(m)

[1, 1]

In [11]:
# test 10
# s1 terminal state
# [1, 1]
m = [
[1, 1],
[0, 0]
]
solution(m)

[1, 1]

In [12]:
# test 11
# s1 terminal state
# [1, ]
m = [
[0, 0],
[0, 0]
]
solution(m)

[1, 0, 1]

In [13]:
# test 12
# s1 terminal state
# [1, ]
m = [
[0,],
]
solution(m)

[1, 1]

In [14]:
# test 13
# s1 terminal state
# [1, ]
m = [
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]
]
solution(m)

[1, 0, 0, 1]

In [15]:
# test 14
# test 5 for google
# a = [1, 2, 3, 4, 5, 15]
m = [
    [0, 0, 12, 0, 15, 0, 0, 0, 1, 8],
    [0, 0, 60, 0, 0, 7, 13, 0, 0, 0],
    [0, 15, 0, 8, 7, 0, 0, 1, 9, 0],
    [23, 0, 0, 0, 0, 1, 0, 0, 0, 0], 
    [37, 35, 0, 0, 0, 0, 3, 21, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
]
solution(m)

[1, 2, 3, 4, 5, 15]

In [16]:
# test 15
# test 3 for google
#a = [1, 2, 3]
m = [
    [1, 2, 3, 0, 0, 0], 
    [4, 5, 6, 0, 0, 0], 
    [7, 8, 9, 1, 0, 0],
    [0, 0, 0, 0, 1, 2],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0]
]
solution(m)

[1, 2, 3]

In [17]:
# TEST 6
# s5, s6, s7, s8, s9
#a = [4, 5, 5, 4, 2, 20]
m = [
    [ 0,  7,  0, 17,  0,  1,  0,  5,  0,  2],
    [ 0,  0, 29,  0, 28,  0,  3,  0, 16,  0],
    [ 0,  3,  0,  0,  0,  1,  0,  0,  0,  0], # s2
    [48,  0,  3,  0,  0,  0, 17,  0,  0,  0], # s3
    [ 0,  6,  0,  0,  0,  1,  0,  0,  0,  0], # s4
    
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # s5
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # s6
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # s7
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # s8
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]] # s9
solution(m)

[4, 5, 5, 4, 2, 20]

In [18]:
#https://pastebin.com/HAiEwnD2
# TEST 7
#m = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
#a = [1, 1, 1, 1, 1, 5]
 
# TEST 8
#m = [[1, 1, 1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
#a = [2, 1, 1, 1, 1, 6]
 
# TEST 9
#m = [[0, 86, 61, 189, 0, 18, 12, 33, 66, 39], [0, 0, 2, 0, 0, 1, 0, 0, 0, 0], [15, 187, 0, 0, 18, 23, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
#a = [6, 44, 4, 11, 22, 13, 100]
 
# TEST 10
#m = [[0, 0, 0, 0, 3, 5, 0, 0, 0, 2], [0, 0, 4, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 4, 4, 0, 0, 0, 1, 1], [13, 0, 0, 0, 0, 0, 2, 0, 0, 0], [0, 1, 8, 7, 0, 0, 0, 1, 3, 0], [1, 7, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
#a = [1, 1, 1, 2, 5]

In [19]:
# TEST 7
m = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
#a = [1, 1, 1, 1, 1, 5]
solution(m)

[1, 1, 1, 1, 1, 5]

In [20]:
# TEST 8
m = [[1, 1, 1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
#a = [2, 1, 1, 1, 1, 6]
solution(m)

[2, 1, 1, 1, 1, 6]

In [21]:
# TEST 9
m = [[0, 86, 61, 189, 0, 18, 12, 33, 66, 39], [0, 0, 2, 0, 0, 1, 0, 0, 0, 0], [15, 187, 0, 0, 18, 23, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
#a = [6, 44, 4, 11, 22, 13, 100]
solution(m)

[6, 44, 4, 11, 22, 13, 100]

In [22]:
# TEST 10
m = [[0, 0, 0, 0, 3, 5, 0, 0, 0, 2], [0, 0, 4, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 4, 4, 0, 0, 0, 1, 1], [13, 0, 0, 0, 0, 0, 2, 0, 0, 0], [0, 1, 8, 7, 0, 0, 0, 1, 3, 0], [1, 7, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
#a = [1, 1, 1, 2, 5]
solution(m)

[1, 1, 1, 2, 5]